# Create Submission File for Kaggle

### Load Test Data

In [1]:
from src.data.data_loader import get_loader

raw_data = get_loader(dataset_type='test')

In [ ]:
data = get_loader(dataset_type='test')

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f6ade423a40>>
Traceback (most recent call last):
  File "/home/julia/projects/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


## Process data
I took these functions from BrisT1DDataLoader._process_raw_data

In [ ]:
from src.data.data_cleaner import clean_brist1d_test_data
from src.data.data_transforms import (
    create_datetime_index,
    create_cob_and_carb_availability_cols,
    create_iob_and_ins_availability_cols,
    ensure_regular_time_intervals,
)

processed_data = clean_brist1d_test_data(raw_data)

In [ ]:
processed_data['p01'].keys()

dict_keys(['p01_8459', 'p01_8460', 'p01_8461', 'p01_8462', 'p01_8463', 'p01_8464', 'p01_8465', 'p01_8466', 'p01_8467', 'p01_8468', 'p01_8469', 'p01_8470', 'p01_8471', 'p01_8472', 'p01_8473', 'p01_8474', 'p01_8475', 'p01_8476', 'p01_8477', 'p01_8478', 'p01_8479', 'p01_8480', 'p01_8481', 'p01_8482', 'p01_8483', 'p01_8484', 'p01_8485', 'p01_8486', 'p01_8487', 'p01_8488', 'p01_8489', 'p01_8490', 'p01_8491', 'p01_8492', 'p01_8493', 'p01_8494', 'p01_8495', 'p01_8496', 'p01_8497', 'p01_8498', 'p01_8499', 'p01_8500', 'p01_8501', 'p01_8502', 'p01_8503', 'p01_8504', 'p01_8505', 'p01_8506', 'p01_8507', 'p01_8508', 'p01_8509', 'p01_8510', 'p01_8511', 'p01_8512', 'p01_8513', 'p01_8514', 'p01_8515', 'p01_8516', 'p01_8517', 'p01_8518', 'p01_8519', 'p01_8520', 'p01_8521', 'p01_8522', 'p01_8523', 'p01_8524', 'p01_8525', 'p01_8526', 'p01_8527', 'p01_8528', 'p01_8529', 'p01_8530', 'p01_8531', 'p01_8532', 'p01_8533', 'p01_8534', 'p01_8535', 'p01_8536', 'p01_8537', 'p01_8538', 'p01_8539', 'p01_8540', 'p01_

In [ ]:
row1 = processed_data['p01']['p01_8459']

In [ ]:
row1

,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00,p_num
0,00:50:00,NaN,0.0083,NaN,59.7,0.0,5.30,NaN,p01
1,00:55:00,9.2,0.0083,NaN,55.6,NaN,5.30,NaN,p01
2,01:00:00,NaN,0.0083,NaN,58.2,0.0,5.00,NaN,p01
3,01:05:00,NaN,0.0083,NaN,59.3,0.0,5.20,NaN,p01
4,01:10:00,10.2,0.0083,NaN,58.0,0.0,6.25,NaN,p01
...,...,...,...,...,...,...,...,...,...
67,06:25:00,NaN,0.0417,NaN,53.2,0.0,4.80,NaN,p01
68,06:30:00,9.4,0.0417,NaN,55.6,0.0,4.80,NaN,p01
69,06:35:00,NaN,0.0417,NaN,55.1,NaN,4.80,NaN,p01
70,06:40:00,NaN,0.0417,NaN,54.9,NaN,4.90,NaN,p01


In [ ]:
row1 = create_datetime_index(row1)
row1 = ensure_regular_time_intervals(row1)
row1 = create_cob_and_carb_availability_cols(row1)
row1 = create_iob_and_ins_availability_cols(row1)

In [ ]:
row1

,datetime,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00,p_num,id,cob,carb_availability,insulin_availability,iob
0,2025-01-01 00:50:00,00:50:00,NaN,0.0083,NaN,59.7,0.0,5.30,NaN,p01,p01_0,0.0,0.0,0.000000,0.008300
1,2025-01-01 00:55:00,00:55:00,9.2,0.0083,NaN,55.6,NaN,5.30,NaN,p01,p01_1,0.0,0.0,0.000015,0.016599
2,2025-01-01 01:00:00,01:00:00,NaN,0.0083,NaN,58.2,0.0,5.00,NaN,p01,p01_2,0.0,0.0,0.000081,0.024874
3,2025-01-01 01:05:00,01:05:00,NaN,0.0083,NaN,59.3,0.0,5.20,NaN,p01,p01_3,0.0,0.0,0.000209,0.033087
4,2025-01-01 01:10:00,01:10:00,10.2,0.0083,NaN,58.0,0.0,6.25,NaN,p01,p01_4,0.0,0.0,0.000399,0.041194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2025-01-01 06:25:00,06:25:00,NaN,0.0417,NaN,53.2,0.0,4.80,NaN,p01,p01_67,0.0,0.0,0.158103,2.496160
68,2025-01-01 06:30:00,06:30:00,9.4,0.0417,NaN,55.6,0.0,4.80,NaN,p01,p01_68,0.0,0.0,0.157754,2.427293
69,2025-01-01 06:35:00,06:35:00,NaN,0.0417,NaN,55.1,NaN,4.80,NaN,p01,p01_69,0.0,0.0,0.156703,2.358868
70,2025-01-01 06:40:00,06:40:00,NaN,0.0417,NaN,54.9,NaN,4.90,NaN,p01,p01_70,0.0,0.0,0.155073,2.291334


In [ ]:
# import pandas as pd
from src.tuning.benchmark import impute_missing_values


def reduce_features(df):
    # Make sure index is set to datetime
    p_df = df.iloc[:]
    # p_df.set_index("datetime", inplace=True)
    # p_df = p_df.asfreq('5T', method='pad')

    # Reduce features
    y_feature = ["bg-0:00"]
    x_features = [
        # "hr-0:00",
        "steps-0:00",
        # "cals-0:00",
        "cob",
        "carb_availability",
        "insulin_availability",
        "iob",
    ]
    p_df = p_df[x_features + y_feature]

    # Impute with default methods
    p_df = impute_missing_values(p_df, columns=x_features)
    p_df = impute_missing_values(p_df, columns=y_feature)

    y, X = p_df[y_feature], p_df[x_features]
    return y, X

In [ ]:
y, X = reduce_features(row1)

In [ ]:
y

,bg-0:00
0,9.200000
1,9.200000
2,9.533333
3,9.866667
4,10.200000
...,...
67,9.675000
68,9.400000
69,9.466667
70,9.533333


In [ ]:
X

,steps-0:00,cob,carb_availability,insulin_availability,iob
0,0.0,0.0,0.0,0.000000,0.008300
1,0.0,0.0,0.0,0.000015,0.016599
2,0.0,0.0,0.0,0.000081,0.024874
3,0.0,0.0,0.0,0.000209,0.033087
4,0.0,0.0,0.0,0.000399,0.041194
...,...,...,...,...,...
67,0.0,0.0,0.0,0.158103,2.496160
68,0.0,0.0,0.0,0.157754,2.427293
69,0.0,0.0,0.0,0.156703,2.358868
70,0.0,0.0,0.0,0.155073,2.291334


## Tiny Time Mixer Forecast

### Load model with best params

In [ ]:
from src.tuning.load_estimators import load_all_forecasters, get_estimator

# This loads all sktime forecasters to be accessible by their class name -> don't need to run again
forecasters = load_all_forecasters()

In [ ]:
from src.utils.config_loader import load_yaml_config

# Set model as sktime forecaster class name
model = "TinyTimeMixerForecaster"
forecaster_class = get_estimator(forecasters, model)

# Set relative path to config yaml file with params for model
config_path = "../../src/models/configs/TinyTimeMixerForecaster.yaml"

params = load_yaml_config(config_path)

forecaster = forecaster_class(**params)

In [ ]:
from sktime.forecasting.base import ForecastingHorizon
fh=ForecastingHorizon(np.arange(1,13), is_relative=True)

forecaster.fit(y=y, X=X, fh = fh)

TinyTimeMixerForecaster(training_args={'num_train_epochs': 10000000,
                                       'output_dir': 'test_output',
                                       'per_device_train_batch_size': 32})

In [ ]:
y_pred = forecaster.predict(fh=np.arange(1,13)) # what to use for X here?
y_pred

,bg-0:00
72,12.051710
73,12.082664
74,11.672007
75,11.979145
76,11.944000
77,11.632549
78,11.216318
79,11.190398
80,11.487564
81,11.398602


### NaiveForecaster

In [ ]:
forecaster_class = get_estimator(forecasters, 'NaiveForecaster')                
forecaster = forecaster_class(strategy="last")
forecaster.fit(y=y, X=X, fh=fh)

NaiveForecaster()

In [ ]:
forecaster.predict(fh=fh)

,bg-0:00
72,9.6
73,9.6
74,9.6
75,9.6
76,9.6
77,9.6
78,9.6
79,9.6
80,9.6
81,9.6


### script to loop through all test data rows

In [ ]:
from src.tuning.load_estimators import load_all_forecasters, get_estimator
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os 

# This loads all sktime forecasters to be accessible by their class name -> don't need to run again
forecasters = load_all_forecasters()

def create_submission_file(patient_dfs, model, params={}, fh=np.arange(1,13), save_path='.'):
    results = []

    for pid in tqdm(patient_dfs, desc='Patients'):
        for row_id in tqdm(patient_dfs[pid], desc=f"Rows for {pid}", leave=False):
            df = patient_dfs[pid][row_id]
            try:
                df = create_datetime_index(df)
                df = ensure_regular_time_intervals(df)
                df = create_cob_and_carb_availability_cols(df)
                df = create_iob_and_ins_availability_cols(df)

                y, X = reduce_features(df)
                
                forecaster_class = get_estimator(forecasters, model)                
                forecaster = forecaster_class(**params)

                forecaster.fit(y=y, X=X, fh=fh)
                y_pred = forecaster.predict(fh=fh)

                predicted_value = y_pred.iloc[-1].item()

                results.append({"id": row_id, "bg+1:00": predicted_value})

            except Exception as e:
                print(f"Failed to forecast {row_id} ({pid}): {e}")

    # save to file
    results_df = pd.DataFrame(results)
    filename = f"{model}_submission.csv"

    os.makedirs(save_path, exist_ok=True)
    file_path = os.path.join(save_path, filename)

    results_df.to_csv(file_path, index=False)
    return results_df

In [ ]:
create_submission_file(processed_data, 'NaiveForecaster', {"strategy": "last"}, save_path='../competition_submission')

Patients:   0%|          | 0/15 [00:00<?, ?it/s]

Rows for p01:   0%|          | 0/244 [00:00<?, ?it/s]

Rows for p02:   0%|          | 0/227 [00:00<?, ?it/s]

Rows for p04:   0%|          | 0/258 [00:00<?, ?it/s]

Rows for p05:   0%|          | 0/276 [00:00<?, ?it/s]

...

Rows for p19:   0%|          | 0/246 [00:00<?, ?it/s]

Rows for p21:   0%|          | 0/236 [00:00<?, ?it/s]

Rows for p22:   0%|          | 0/201 [00:00<?, ?it/s]

Rows for p24:   0%|          | 0/261 [00:00<?, ?it/s]

,id,bg+1:00
0,p01_8459,9.6
1,p01_8460,4.6
2,p01_8461,8.0
3,p01_8462,9.9
4,p01_8463,5.3
...,...,...
3639,p24_256,6.3
3640,p24_257,10.4
3641,p24_258,6.9
3642,p24_259,8.7


In [ ]:
from src.utils.config_loader import load_yaml_config

config_path = "../../src/models/configs/TinyTimeMixerForecaster.yaml"
params = load_yaml_config(config_path)

create_submission_file(processed_data, 'TinyTimeMixerForecaster', params, save_path='../competition_submission')


Rows for p01:   0%|          | 0/244 [00:00<?, ?it/s]

,id,bg+1:00
0,p01_8459,11.307776
1,p01_8460,5.439600
2,p01_8461,7.188557
3,p01_8462,8.037812
4,p01_8463,6.701509
...,...,...
239,p01_8698,5.816570
240,p01_8699,5.001166
241,p01_8700,14.397934
242,p01_8701,12.422853
